In [8]:
import keras
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing import image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,Activation
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [9]:
from PIL import Image
cur_path = r'C:\Users\chris\Documents\Git_repo\GTSRB_TF\GTSRB---German-Traffic-Sign-Recognition\gtsrb-german-traffic-sign'

In [10]:
data = []
labels = []
classes = 43
 

#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)

    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
    
            #data.append(image)
            #labels.append(i)
            data.append([image,i]) #appending all value together 
        except:
            print("Error loading image")

In [11]:
random.shuffle(data)

In [12]:
print(len(data))

39209


In [13]:
x = []
y = []

for features,label in data:
    x.append(features)
    y.append(label)

In [14]:
#Converting lists into numpy arrays
x = np.array(x)
y = np.array(y)

In [15]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [16]:
X_train = X_train/255.0
X_val = X_val/255.0

In [10]:
#x = np.array(x).reshape(-1, 30, 30, 1) 

In [17]:
print("Shape of train images is:", X_train.shape)
print("Shape of labels is:", y_train.shape)

Shape of train images is: (31367, 30, 30, 3)
Shape of labels is: (31367,)


In [18]:
#Building the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(30, 30, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(43))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

c:\Users\chris\Documents\Git_repo\GTSRB_TF\GTSRB---German-Traffic-Sign-Recognition\.conda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

model.fit(aug.flow(X_train, y_train, batch_size=32), epochs=15, validation_data=(X_val, y_val))

model.save('my_model2.h5')  # always save your weights after training or during training

Epoch 1/15


c:\Users\chris\Documents\Git_repo\GTSRB_TF\GTSRB---German-Traffic-Sign-Recognition\.conda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


981/981 ━━━━━━━━━━━━━━━━━━━━ 27s 26ms/step - accuracy: 0.1657 - loss: 3.0908 - val_accuracy: 0.5416 - val_loss: 1.3512
Epoch 2/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 24s 25ms/step - accuracy: 0.4828 - loss: 1.5750 - val_accuracy: 0.7456 - val_loss: 0.7753
Epoch 3/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 22s 23ms/step - accuracy: 0.6280 - loss: 1.0794 - val_accuracy: 0.8407 - val_loss: 0.4823
Epoch 4/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 22s 23ms/step - accuracy: 0.7433 - loss: 0.7478 - val_accuracy: 0.9220 - val_loss: 0.2739
Epoch 5/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - accuracy: 0.8212 - loss: 0.5344 - val_accuracy: 0.9607 - val_loss: 0.1587
Epoch 6/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 23s 23ms/step - accuracy: 0.8651 - loss: 0.4073 - val_accuracy: 0.9726 - val_loss: 0.0990
Epoch 7/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 23s 24ms/step - accuracy: 0.8952 - loss: 0.3161 - val_accuracy: 0.9802 - val_loss: 0.0844
Epoch 8/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.9176 - loss: 0.2582 - val_accurac

In [20]:
#testing accuracy on test dataset
from sklearn.metrics import accuracy_score

y_test = pd.read_csv('gtsrb-german-traffic-sign/Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))

X_test=np.array(data)

#pred = model.predict_classes(X_test)  

pred = np.argmax(model.predict(X_test), axis=-1)

#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))

395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
0.920110847189232


#### Here we are testing our model on our saved model also we can test on our 15 epoch data result we got.

In [24]:
from keras.models import load_model
model = load_model('my_model2.h5')

In [30]:
path = r'C:\Users\chris\Documents\Git_repo\GTSRB_TF\GTSRB---German-Traffic-Sign-Recognition\Test\12614.png'

In [31]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
traffic_signs = {
    0: 'Speed limit (20km/h)',
    1: 'Speed limit (30km/h)',
    2: 'Speed limit (50km/h)',
    3: 'Speed limit (60km/h)',
    4: 'Speed limit (70km/h)',
    5: 'Speed limit (80km/h)',
    6: 'End of speed limit (80km/h)',
    7: 'Speed limit (100km/h)',
    8: 'Speed limit (120km/h)',
    9: 'No passing',
    10: 'No passing vehicles over 3.5 tons',
    11: 'Right-of-way at intersection',
    12: 'Priority road',
    13: 'Yield',
    14: 'Stop',
    15: 'No vehicles',
    16: 'Vehicles over 3.5 tons prohibited',
    17: 'No entry',
    18: 'General caution',
    19: 'Dangerous curve left',
    20: 'Dangerous curve right',
    21: 'Double curve',
    22: 'Bumpy road',
    23: 'Slippery road',
    24: 'Road narrows on the right',
    25: 'Road work',
    26: 'Traffic signals',
    27: 'Pedestrians',
    28: 'Children crossing',
    29: 'Bicycles crossing',
    30: 'Beware of ice/snow',
    31: 'Wild animals crossing',
    32: 'End speed + passing limits',
    33: 'Turn right ahead',
    34: 'Turn left ahead',
    35: 'Ahead only',
    36: 'Go straight or right',
    37: 'Go straight or left',
    38: 'Keep right',
    39: 'Keep left',
    40: 'Roundabout mandatory',
    41: 'End of no passing',
    42: 'End no passing vehicles > 3.5 tons'
}

# Load and preprocess the image
img = load_img(path, target_size=(30, 30))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

# Predict the label for the image
classes = np.argmax(model.predict(images, batch_size=32), axis=-1)

# Print the traffic sign description
print(traffic_signs.get(classes[0], "Unknown sign"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Speed limit (20km/h)
